In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import nltk
import re
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.manifold import TSNE 
import sqlite3

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
con = sqlite3.connect("C:/Users/israe/OneDrive/Documentos/DB/banco_scraping_olx.db")
df = pd.read_sql_query("SELECT * FROM anuncios_resumo", con)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
def build_corpus(data):
  stop_words = set(stopwords.words('portuguese'))
  corpus = []
  for col in ["titulo_anuncio"]:
    for sentence in data[col].iteritems():
      word_list = re.split('[\n#$&()./:=_*?!<>| ,+-;]', sentence[1])
      filtered = [w.lower() for w in word_list if not w.lower() in stop_words]
      corpus.append(list(filter(None, filtered)))
  return corpus

In [ ]:
df

In [ ]:
mask = df["categoria_completa"].str.contains("Imóveis|Autos e peças|Eletrônicos e celulares", regex = True)

In [ ]:
df2 = df[mask]

In [ ]:
df2

In [ ]:
corpus = build_corpus(df)

In [ ]:
model = Word2Vec(corpus, min_count = 5, max_final_vocab=500000, vector_size = 300)

In [ ]:
model = Word2Vec.load("word2vec.model")

In [ ]:
model.save("word2vec.model")

In [ ]:
corpus.clear()

In [ ]:
del(corpus)

In [ ]:
del(mask)

In [ ]:
del(con)

In [ ]:
del(df)

In [ ]:
del(df2)

In [ ]:
corpus2 = build_corpus(df2)

In [ ]:
tamC2 = len(corpus2)

In [ ]:
len(df.index)

In [ ]:
print(model.wv.key_to_index)

In [ ]:
vec = np.zeros((tamC2, 300))
for i in range(tamC2):
    for w in corpus2[i]:
        if w in model.wv.key_to_index:
            vec[i] += model.wv[w]
    vec[i] = np.mean(vec[i], axis=0)

In [ ]:
vec

In [ ]:
corpus2

In [ ]:
tsne = TSNE(n_components=2, random_state=0) 

In [ ]:
Y = tsne.fit_transform(vec[:,:1000]) 

In [ ]:
plt.scatter(Y[:, 0], Y[:, 1]) 
for label, x, y in zip(vocabulary, Y[:, 0], Y[:, 1]): 
  plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points') 
  plt.show() 